In [1]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from imutils import face_utils
font = cv2.FONT_HERSHEY_SIMPLEX

In [3]:
import os

def get_files(path):
    return os.listdir(path)

cascPath = "/Users/telekenesis/Husky Gdrive/Coursework/7. SML_EE/Project/image/haarcascade_frontalface_default.xml"

def return_bbx(image):
    faceCascade = cv2.CascadeClassifier(cascPath)
    faces = faceCascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, flags=cv2.CASCADE_SCALE_IMAGE)
    return faces

In [4]:
Celebs = pd.read_csv("/Users/telekenesis/Husky Gdrive/Coursework/7. SML_EE/Project/lfw_allnames.csv")
Celebs = Celebs[Celebs['images']>50]
Celebs

,name,images
373,Ariel_Sharon,77
1047,Colin_Powell,236
1404,Donald_Rumsfeld,121
1871,George_W_Bush,530
1892,Gerhard_Schroeder,109
2175,Hugo_Chavez,71
2288,Jacques_Chirac,52
2468,Jean_Chretien,55
2682,John_Ashcroft,53
2941,Junichiro_Koizumi,60


In [6]:
Dataset_path = "/Users/telekenesis/Husky Gdrive/Coursework/7. SML_EE/Project/Data"

In [9]:
X = []
Y = []

for _, [name,__] in Celebs.iterrows():
    celeb_path = Dataset_path+'/'+name+'/'
    
    images_paths = get_files(celeb_path)
    for image_path in images_paths:
        image = cv2.imread(celeb_path+image_path,1)
        faces = return_bbx(image)
        if len(faces) == 1:
            (x,y,w,h) = faces[0]
            cropped = image[x:x+w, y:y+h]
            dim = (64, 64)
            resized = cv2.resize(cropped, dim, interpolation = cv2.INTER_AREA)
            image = np.array(resized).astype("float32")
            X.append(image)
            Y.append(name)



In [10]:
X_data = np.array(X)
Y_data = np.array(Y)

In [11]:
from collections import Counter

Counter(Y_data)

Counter({'Ariel_Sharon': 70,
         'Colin_Powell': 214,
         'Donald_Rumsfeld': 109,
         'George_W_Bush': 480,
         'Gerhard_Schroeder': 106,
         'Hugo_Chavez': 67,
         'Jacques_Chirac': 51,
         'Jean_Chretien': 51,
         'John_Ashcroft': 47,
         'Junichiro_Koizumi': 51,
         'Serena_Williams': 48,
         'Tony_Blair': 138})

In [12]:
from imblearn.under_sampling import NearMiss

X_data_reshaped = X_data.reshape(X_data.shape[0], X_data.shape[1] * X_data.shape[2] * X_data.shape[3])
undersample = NearMiss(version=1)
X_resampled, Y_resampled = undersample.fit_resample(X_data_reshaped,Y_data)
# X_data = X_resampled.reshape(X_resampled.shape[0], X_data.shape[1], X_data.shape[2], X_data.shape[3])
Y_data = Y_resampled
counter = Counter(Y_data)
print(counter)

del X_data_reshaped
del undersample
# del X_resampled
del Y_resampled
del counter

Counter({'Ariel_Sharon': 47, 'Colin_Powell': 47, 'Donald_Rumsfeld': 47, 'George_W_Bush': 47, 'Gerhard_Schroeder': 47, 'Hugo_Chavez': 47, 'Jacques_Chirac': 47, 'Jean_Chretien': 47, 'John_Ashcroft': 47, 'Junichiro_Koizumi': 47, 'Serena_Williams': 47, 'Tony_Blair': 47})


In [13]:

a = Y_data
n_classes = len(set(a))
n_classes

12

In [14]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(sparse=False)

Y_data_ohe = enc.fit_transform(Y_data.reshape(-1,1))

In [15]:
import os
import cv2
import random
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from datetime import datetime
from sklearn.utils import shuffle

%load_ext tensorboard

ModuleNotFoundError: No module named 'tqdm'